In [ ]:
!wget -nc https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -n ml-latest-small.zip

--2024-11-29 10:15:58--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  3.26MB/s    in 0.3s    

2024-11-29 10:15:59 (3.26 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
# Install necessary libraries
!pip install numpy pandas surprise scikit-learn streamlit --quiet
!pip install --upgrade pyngrok  # To expose Streamlit app via ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

from difflib import get_close_matches

from IPython.display import Image, display, HTML

import requests

In [ ]:
pd.set_option('display.max_columns', None)

## **Data Loading**

In [ ]:
# tags = pd.read_csv('ml-latest-small/tags.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
links = pd.read_csv('links_with_poster.csv')

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
# Spliting the genres column into multiple binary columns
genres = set("|".join(movies["genres"].unique()).split("|"))

# Removing the "(no genres listed)" genre
genres.discard("(no genres listed)")

for genre in genres:
    movies[genre] = movies["genres"].apply(lambda x: 1 if genre in x else 0)

movies.drop(columns = ['genres', 'IMAX'], axis=1, inplace=True) # Dropping the IMAX column becasue it is not a genre

In [ ]:
movies

,movieId,title,Drama,Fantasy,Animation,Horror,Thriller,War,Mystery,Children,Action,Documentary,Sci-Fi,Crime,Romance,Western,Musical,Comedy,Film-Noir,Adventure
0,1,Toy Story (1995),0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
1,2,Jumanji (1995),0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,3,Grumpier Old Men (1995),0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,4,Waiting to Exhale (1995),1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
9738,193583,No Game No Life: Zero (2017),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9739,193585,Flint (2017),1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
# Merge the ratings and movies dataframes
merged_data = pd.merge(ratings, movies, on="movieId")
merged_data = merged_data.drop(["timestamp"], axis=1)
merged_data

# <b>Content based filtering</b>

In [ ]:
movies["genres"] = movies.iloc[:, 4:].dot(movies.columns[4:] + " ")
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(movies["genres"])

content_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
user_item_matrix = merged_data.pivot(index="userId", columns="movieId", values="rating").fillna(0)

In [ ]:
# user_item_matrix

# **Collaborative Filtering**

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merged_data[["userId", "movieId", "rating"]], reader)
algo = SVD()
cross_validate(algo, data, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8748  0.8746  0.8728  0.8695  0.8747  0.8733  0.0020  
MAE (testset)     0.6713  0.6717  0.6710  0.6676  0.6740  0.6711  0.0020  
Fit time          4.07    3.47    1.75    1.85    2.68    2.76    0.90    
Test time         0.36    0.13    0.29    0.18    0.21    0.23    0.08    


{'test_rmse': array([0.87477764, 0.87459876, 0.87282142, 0.86948516, 0.87471521]),
 'test_mae': array([0.67128981, 0.67167512, 0.67101764, 0.66762563, 0.67401785]),
 'fit_time': (4.06943154335022,
  3.468261480331421,
  1.754732370376587,
  1.846024513244629,
  2.6763596534729004),
 'test_time': (0.35515928268432617,
  0.12664031982421875,
  0.28831005096435547,
  0.18245196342468262,
  0.2147049903869629)}

In [ ]:
def predict_ratings(user_id, movie_id):
    return algo.predict(user_id, movie_id).est

In [ ]:
# Merge tmdbId with movies DataFrame
movies = movies.merge(links[['movieId', 'poster_url']], on='movieId', how='left')

from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv('API_KEY')

# Function to fetch poster URL using tmdbId
def fetch_poster_with_tmdb_id(tmdb_id):
    """
    Fetch the poster URL using tmdbId from the TMDB API.

    Args:
        tmdb_id (int): TMDB ID of the movie.

    Returns:
        str: URL of the movie poster or None if not found.
    """
    if pd.isna(tmdb_id):
        return None  # If no tmdbId is available

    api_key = API_KEY  # Replace with your TMDB API key
    base_url = f"https://api.themoviedb.org/3/movie/{int(tmdb_id)}"
    query = {"api_key": api_key}

    try:
        response = requests.get(base_url, params=query)
        response_data = response.json()
        if "poster_path" in response_data and response_data["poster_path"]:
            return f"https://image.tmdb.org/t/p/w500{response_data['poster_path']}"
        else:
            return None
    except Exception as e:
        print(f"Error fetching poster for TMDB ID {tmdb_id}: {e}")
        return None

In [ ]:
# links['poster_url'] = links['tmdbId'].apply(fetch_poster_with_tmdb_id)

In [ ]:
# links.to_csv('links_with_poster.csv', index=False)

In [ ]:
movies

,movieId,title,Drama,Fantasy,Animation,Horror,Thriller,War,Mystery,Children,Action,Documentary,Sci-Fi,Crime,Romance,Western,Musical,Comedy,Film-Noir,Adventure,genres
0,1,Toy Story (1995),0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,Animation Children Comedy Adventure
1,2,Jumanji (1995),0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,Children Adventure
2,3,Grumpier Old Men (1995),0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,Romance Comedy
3,4,Waiting to Exhale (1995),1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,Romance Comedy
4,5,Father of the Bride Part II (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,Animation Action Comedy
9738,193583,No Game No Life: Zero (2017),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Animation Comedy
9739,193585,Flint (2017),1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,
9740,193587,Bungo Stray Dogs: Dead Apple (2018),0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Animation Action


In [ ]:
# links = movies['tmdbId'].head(10).apply(fetch_poster_with_tmdb_id)

In [ ]:
# links

In [ ]:
# Create a mapping of movieId to its index in the dataset
movieId_to_index = {movie_id: idx for idx, movie_id in enumerate(movies["movieId"])}
index_to_movieId = {idx: movie_id for movie_id, idx in movieId_to_index.items()}

In [ ]:
def recommend_movies(movie_name, num_recommendations=5, userId=None, genre=False):
    """
    Recommend movies based on input movie name.
    - For existing users: Hybrid recommendation.
    - For new users: Content-based recommendation.

    Args:
        movie_name (str): Name of the movie.
        num_recommendations (int): Number of recommendations. Defaults to 5.
        userId (int, optional): ID of the user. Defaults to None (assumes new user).
        genre (bool): Whether to include genres in the output. Defaults to False.

    Returns:
        DataFrame: Recommended movies with scores (and genres if genre=True) and poster links.
    """
    # Check if movie exists, else find the closest match
    if movie_name not in movies["title"].values:

        # Find the closest match to the searched movie name
        close_match = get_close_matches(movie_name, movies["title"].values, n=1, cutoff=0.5)
        if close_match:
            print(f"Movie '{movie_name}' not found. Did you mean '{close_match[0]}'?")
            movie_name = close_match[0]
        else:
            print(f"Movie '{movie_name}' not found, and no close matches are available.")
            return None

    # Map movieId to matrix index, and get the index of the current movie
    movieId = movies[movies["title"] == movie_name]["movieId"].iloc[0]
    current_index = movieId_to_index[movieId]

    # Check if user exists
    if userId and userId in merged_data["userId"].unique():
        print(f"User found! Applying hybrid recommendation for user ID {userId}.")

        # Movies rated by the user
        user_movies = merged_data[merged_data["userId"] == userId]
        watched_movie_ids = user_movies["movieId"].tolist()

        # Hybrid recommendation, combining content-based and collaborative filtering
        recommended_movies = []
        for candidate_movie_id in movies["movieId"]:
            if candidate_movie_id not in watched_movie_ids:
                candidate_index = movieId_to_index[candidate_movie_id]
                cb_score = content_similarity[current_index, candidate_index]
                cf_score = predict_ratings(userId, candidate_movie_id)
                hybrid_score = 0.5 * cb_score + 0.5 * cf_score
                recommended_movies.append((candidate_movie_id, hybrid_score))
    else:
        print("No user data found. Applying content-based recommendation.")

        # Get content similarity scores
        similarity_scores = content_similarity[current_index]
        recommended_movies = [
            (index_to_movieId[idx], score) for idx, score in enumerate(similarity_scores)
        ]

    # Sorting the recommended movies by score in descending order
    recommendations = sorted(recommended_movies, key=lambda x: x[1], reverse=True)[:num_recommendations]

    # Fetching movie details for recommendations
    recommended_movies_df = pd.DataFrame(recommendations, columns=["movieId", "score"])
    recommended_movies_df = recommended_movies_df.merge(movies, on="movieId", how="left")
    recommended_movies_df = recommended_movies_df.merge(links, on="movieId", how="left")

    if genre:
        result_df = recommended_movies_df[["title", "score", "genres", "poster_url"]]
    else:
        result_df = recommended_movies_df[["title", "score", "poster_url"]]

    return result_df


In [ ]:
movie_to_search = "Toy Story"
recommendations = recommend_movies(movie_to_search, num_recommendations=10, genre=True)
recommendations

Movie 'Toy Story' not found. Did you mean 'Toy Story (1995)'?
No user data found. Applying content-based recommendation.


,title,score,genres,poster_url
0,Toy Story (1995),1.0,Animation Children Comedy Adventure,https://image.tmdb.org/t/p/w500/uXDfjJbdP4ijW5...
1,"American Tail, An (1986)",1.0,Animation Children Comedy Adventure,https://image.tmdb.org/t/p/w500/qbmRSCMMIlBboo...
2,Antz (1998),1.0,Animation Children Comedy Adventure,https://image.tmdb.org/t/p/w500/lWPjxbUMpAHFkJ...
3,"Bug's Life, A (1998)",1.0,Animation Children Comedy Adventure,https://image.tmdb.org/t/p/w500/Ah3J9OJVc2CNCu...
4,Toy Story 2 (1999),1.0,Animation Children Comedy Adventure,https://image.tmdb.org/t/p/w500/2MFIhZAW0CVlEQ...
5,"Adventures of Rocky and Bullwinkle, The (2000)",1.0,Animation Children Comedy Adventure,https://image.tmdb.org/t/p/w500/xCFSsftt2rglC8...
6,"Emperor's New Groove, The (2000)",1.0,Animation Children Comedy Adventure,https://image.tmdb.org/t/p/w500/wwbgkXQBEKtnyI...
7,"Monsters, Inc. (2001)",1.0,Animation Children Comedy Adventure,https://image.tmdb.org/t/p/w500/wFSpyMsp7H0ttE...
8,Jimmy Neutron: Boy Genius (2001),1.0,Animation Children Comedy Adventure,https://image.tmdb.org/t/p/w500/pUwdquA6Bf3Gq8...
9,Ice Age (2002),1.0,Animation Children Comedy Adventure,https://image.tmdb.org/t/p/w500/gLhHHZUzeseRXS...


In [ ]:
# This code is generated by ChatGPT, bcz i don't know CSS
html = '<div style="display: flex; flex-wrap: wrap;">'
for index, row in recommendations.iterrows():
    html += f'<div style="margin: 10px; width: 18%; text-align: center;">'  # Adjust the width as per the number of columns
    html += f'<img src="{row["poster_url"]}" alt="{row["title"]}" style="width: 100%; height: auto;">'
    html += f'<p>{row["title"]}</p>'
    html += f'</div>'
html += '</div>'

# Display the HTML
display(HTML(html))

In [ ]:
# Example 2: Guest user (no user data)
movie_to_search = "Zombee"
recommendations = recommend_movies(movie_to_search, num_recommendations=10, genre=True)
print(f"Recommendations for '{movie_to_search}' (new user):")
recommendations

Movie 'Zombee' not found. Did you mean 'Zombeavers (2014)'?
No user data found. Applying content-based recommendation.
Recommendations for 'Zombee' (new user):


,title,score,genres,poster_url
0,Evil Dead II (Dead by Dawn) (1987),1.000000,Horror Action Comedy,https://image.tmdb.org/t/p/w500/natuespVhWAHdb...
1,Surf Nazis Must Die (1987),1.000000,Horror Action Comedy,https://image.tmdb.org/t/p/w500/gVECT5DHulQJyo...
2,Buffy the Vampire Slayer (1992),1.000000,Horror Action Comedy,https://image.tmdb.org/t/p/w500/tcF3S4UdFpuoZE...
3,Versus (2000),1.000000,Horror Action Comedy,https://image.tmdb.org/t/p/w500/dEk6Fpm4lkEBQB...
4,Zombieland (2009),1.000000,Horror Action Comedy,https://image.tmdb.org/t/p/w500/dUkAmAyPVqubSB...
5,Zombeavers (2014),1.000000,Horror Action Comedy,https://image.tmdb.org/t/p/w500/gdb7v5GCfijrCj...
6,Dead Snow 2: Red vs. Dead (2014),1.000000,Horror Action Comedy,https://image.tmdb.org/t/p/w500/bllzic3ydohA9i...
7,Scouts Guide to the Zombie Apocalypse (2015),1.000000,Horror Action Comedy,https://image.tmdb.org/t/p/w500/lUKvvSnjFlazrd...
8,Jaws (1975),0.908468,Horror Action,https://image.tmdb.org/t/p/w500/lxM6kqilAdpdhq...
9,Jaws 3-D (1983),0.908468,Horror Action,https://image.tmdb.org/t/p/w500/kqDXj53F9paqVG...


In [ ]:
html = '<div style="display: flex; flex-wrap: wrap;">'
for index, row in recommendations.iterrows():
    html += f'<div style="margin: 10px; width: 18%; text-align: center;">'  # Adjust the width as per the number of columns
    html += f'<img src="{row["poster_url"]}" alt="{row["title"]}" style="width: 100%; height: auto;">'
    html += f'<p>{row["title"]}</p>'  # Movie title below the image
    html += f'</div>'
html += '</div>'

# Display the HTML
display(HTML(html))

In [ ]:
# Example 3: Guest user (no user data)
movie_to_search = "Harry Potter"
recommendations = recommend_movies(movie_to_search, num_recommendations=5, genre=True)
print(f"Recommendations for '{movie_to_search}' (new user):")
recommendations

Movie 'Harry Potter' not found. Did you mean 'Party Monster (2003)'?
No user data found. Applying content-based recommendation.
Recommendations for 'Harry Potter' (new user):


,title,score,genres,poster_url
0,Get Shorty (1995),1.0,Thriller Crime Comedy,https://image.tmdb.org/t/p/w500/r82SdPhg4fnIcL...
1,Pulp Fiction (1994),1.0,Thriller Crime Comedy,https://image.tmdb.org/t/p/w500/d5iIlFn5s0Imsz...
2,Fargo (1996),1.0,Thriller Crime Comedy,https://image.tmdb.org/t/p/w500/bOMoZnNe5B9XzQ...
3,Freeway (1996),1.0,Thriller Crime Comedy,https://image.tmdb.org/t/p/w500/zUEe514X7gKhjU...
4,"Man Who Knew Too Little, The (1997)",1.0,Thriller Crime Comedy,https://image.tmdb.org/t/p/w500/yVlIZa0u6CTN5L...
